In [51]:

import pymysql
import pandas as pd
import getpass
import pandas as pd
import seaborn as sns
import numpy as np
from numpy.linalg import norm
from collections import defaultdict
import itertools

In [3]:
conn = pymysql.connect(host="mysql",
                       port=3306,user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"),db='mimic2')
cursor = conn.cursor()

Enter MySQL passwd for jovyan········


In [5]:
pd.read_sql('SELECT * from icd9',conn).head()


,subject_id,hadm_id,sequence,code,description
0,56,28766,1,198.3,SECONDARY MALIGNANT NEOPLASM OF BRAIN AND SPIN...
1,56,28766,2,162.8,MALIGNANT NEOPLASM OF OTHER PARTS OF BRONCHUS ...
2,56,28766,3,531.40,CHRONIC OR UNSPECIFIED GASTRIC ULCER WITH HEMO...
3,56,28766,4,276.1,HYPOSMOLALITY AND/OR HYPONATREMIA
4,56,28766,5,428.0,CONGESTIVE HEART FAILURE UNSPECIFIED


In [6]:
icd9_codes = pd.read_sql('SELECT subject_id, code from icd9',conn)
icd9_codes.head()

,subject_id,code
0,56,198.3
1,56,162.8
2,56,531.40
3,56,276.1
4,56,428.0


In [15]:
voc_code = icd9_codes.code.unique()
voc_code.sort()

In [16]:
len(voc_code)

2719

In [18]:
voc_map = dict((voc_code[i], i) for i in range(len(voc_code)))
dim = len(voc_map)

In [24]:
patients = defaultdict(list)
for _,row in icd9_codes.iterrows():
    patients[row["subject_id"]].append(row["code"])

In [26]:
max([len(patients[k]) for k in patients]), np.mean([len(patients[k]) for k in patients])

(308, 13.53733232093141)

In [29]:
def patient2vec(p, vmap):
    v = np.zeros(len(vmap))
    for diag in p:
        v[vmap[diag]] += 1
    return v

In [43]:
norm(patient2vec(patients[56], voc_map))

2.8284271247461903

In [44]:
def cos_sim(v1, v2):
    return np.inner(v1,v2)/(norm(v1)*norm(v2))

In [45]:
icd9_codes["subject_id"].unique()

array([   56,    37,    78, ..., 32627, 32686, 32763])

In [46]:
cos_sim(patient2vec(patients[78], voc_map), patient2vec(patients[32627], voc_map))

0.0

In [48]:
f = lambda x : patient2vec(x, voc_map)

In [50]:
patient_vectors = {p:f(patients[p]) for p in patients}

In [57]:
similarity = [(p1,p2,cos_sim(patient_vectors[p1],patient_vectors[p2])) for p1 in patient_vectors for p2 in patient_vectors]

In [56]:
patient_vectors[56]

array([0., 0., 0., ..., 0., 0., 0.])